## Install vizdoom

In [ ]:
%pip install vizdoom

vizdoom: RL platform, runs very quick

In [ ]:
!cd github & git clone https://github.com/mwydmuch/ViZDoom


In [1]:
# import game env
from vizdoom import *
# for random action
import random
# for sleeping
import time
# create action space for random actions
import numpy as np

In [ ]:
# setup game
game = DoomGame()
game.load_config('github/ViZDoom/scenarios/basic.cfg')
game.init()

In [ ]:
# simple action space without double inputs
actions = np.identity(3, dtype='uint8')

In [ ]:
episodes = 10
for episode in range(episodes):
    game.new_episode()
    while not game.is_episode_finished():
        state = game.get_state()
        img = state.screen_buffer
        info = state.game_variables
        reward = game.make_action(random.choice(actions), 4) # frame skip 4 -> get reward after 4 frames
        print(reward)
        time.sleep(1/50)
    print('Total results:', game.get_total_reward())
    time.sleep(2)

In [ ]:
game.close()

## 2. Wrap in Gym wrapper

In [ ]:
%pip install gym

In [2]:
# import base class
from gym import Env
# import spaces
from gym.spaces import Discrete, Box # Discrete is like range(), Box is like array
# import opencv
import cv2 as cv

In [12]:
# build the vizdoom env
class ViZDoomGym(Env):
    def __init__(self, render=False):
        super().__init__()

        self.game = DoomGame()
        self.game.load_config('github/ViZDoom/scenarios/basic.cfg')

        self.game.set_window_visible(render)

        self.game.init()

        self.observation_space = Box(0, 255, shape=(100, 160, 1), dtype='uint8')
        self.action_space = Discrete(3)

    def step(self, action):
        actions = np.identity(3, dtype='uint8')
        reward = self.game.make_action(actions[action])

        if self.game.get_state(): # interesting line
            state = self.game.get_state()
            img = state.screen_buffer
            img = self.grayscale(img)
            ammo = state.game_variables[0]
            info = {'ammo': ammo}
        else:
            img = np.zeros(self.observation_space.shape)
            info = {}

        done = self.game.is_episode_finished()

        return img, reward, done, info

    def close(self):
        self.game.close()

    def __del__(self):
        self.game.close()
        super().__del__()

    def render():
        pass # handled by vizdoom itself

    def grayscale(self, observation):
        img = cv.cvtColor(np.moveaxis(observation, 0, -1), cv.COLOR_BGR2GRAY)
        # scale image down for performance
        img = cv.resize(img, (160, 100), interpolation=cv.INTER_CUBIC)
        img = np.reshape(img, (100, 160, 1)) # add one dimension
        return img

    def reset(self):
        self.game.new_episode()
        img = self.game.get_state().screen_buffer
        return self.grayscale(img)

In [ ]:
env = ViZDoomGym()

In [ ]:
# verify environment
from stable_baselines3.common import env_checker

In [ ]:
env_checker.check_env(env)

## 3. View step

In [ ]:
from matplotlib import pyplot as plt
img = env.step(1)[0]
plt.imshow(img, cmap='gray')
img.shape

## 4. Setup callback

In [4]:
import os
from stable_baselines3.common.callbacks import BaseCallback

c:\Users\phiga\miniconda3\envs\reinforcement\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [6]:
CHECKPOINT_DIR = './doom/train_basic' # for model weights
LOG_DIR = './doom/log_basic' # for tf logs

In [7]:
callback = TrainAndLoggingCallback(check_freq=33000, save_path=CHECKPOINT_DIR)

## 5. Train the model

In [13]:
from stable_baselines3 import PPO

In [16]:
env = ViZDoomGym()

In [19]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, learning_rate=0.0001, n_steps=2048)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [20]:
model.learn(total_timesteps=100000, callback=callback)

Logging to ./doom/log_basic\PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 144      |
|    ep_rew_mean     | -117     |
| time/              |          |
|    fps             | 73       |
|    iterations      | 1        |
|    time_elapsed    | 27       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 118         |
|    ep_rew_mean          | -73.6       |
| time/                   |             |
|    fps                  | 63          |
|    iterations           | 2           |
|    time_elapsed         | 64          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010898388 |
|    clip_fraction        | 0.0569      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.000135 

## Test model

In [23]:
from stable_baselines3.common.evaluation import evaluate_policy

In [32]:
model = PPO.load(CHECKPOINT_DIR + '/best_model_99000')

In [25]:
env = ViZDoomGym(render=True)

Exception ignored in: <function ViZDoomGym.__del__ at 0x0000019BAB89D630>
Traceback (most recent call last):
  File "C:\Users\phiga\AppData\Local\Temp\ipykernel_16636\2583540790.py", line 39, in __del__
AttributeError: 'super' object has no attribute '__del__'


In [29]:
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=100)
mean_reward

c:\Users\phiga\miniconda3\envs\reinforcement\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


72.92

In [35]:
for episode in range(5):
    obs = env.reset()
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        time.sleep(1/50)
        total_reward += reward
    print(f'Reward of episode {episode+1} is {total_reward}')
    time.sleep(2)
env.close()

ViZDoomIsNotRunningException: Controlled ViZDoom instance is not running or not ready.

not perfoect performance